In [10]:
# Library import
import xml.etree.ElementTree as ET
import spacy
import string
import numpy as np
import json
import random
import pickle
# for auto-correct words in review text
from autocorrect import Speller
spell = Speller()

/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorf

In [ ]:
# loading vocabulary of training data
with open('token_to_index_laptops.json', 'rb') as f:
    token_to_index = pickle.load(f)

max_review_length = 35
max_sent_length = 100

In [ ]:
with open('./dataset/valid_laptops.json', 'rb') as f:
    data = pickle.load(f)

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
# removed punctuations
for i in range(len(data)):
    for j in range(len(data[i])):
        sent = data[i][j][0]

        token_lst = []
        for it in nlp(sent):
            token = it.text
            str = ''
            for ch in token:
                if ch == "'" or ch not in string.punctuation:
                    if ch == "'":
                        str += ' '
                    str += ch
                else:
                    str += ' '
            lst = str.split()
            for token in lst:
                if token in token_to_index:
                    token_lst.append(token)
                else:
                    token_lst.append('oov')

        data[i][j][0] = token_lst

        if data[i][j][1] not in token_to_index:
            data[i][j][1] = 'oov'

        if data[i][j][2] not in token_to_index:
            data[i][j][2] = 'oov'

In [ ]:
non_padded_sent_length = []
for i in range(len(data)):
    non_padded_sent_length.append(len(data[i]))

In [ ]:
# review level padded thing
padded_sent = []
for i in range(max_sent_length):
    padded_sent.append('pad')
padded_review = [padded_sent,'pad','pad',1]

# sentence level and review level padding
padded_data = []
temp_data = data.copy()
for i in range(len(data)):
    for j in range(max_review_length):
        if j < len(data[i]):
            token_lst = data[i][j][0]
            sent = []
            for k in range(max_sent_length):
                if k < len(token_lst):
                    sent.append(token_lst[k])
                else:
                    sent.append('pad')
            temp_data[i][j][0] = sent
        
        else:
            temp_data[i].append(padded_review)

    # inserting a token to acknowledge that after this point, only padded sentences occur
    padded_data.append([temp_data[i],non_padded_sent_length[i]])

In [ ]:
review_indices = []
for i in range(len(padded_data)):
    reviews = []
    for j in range(len(padded_data[i][0])):
        review = []
        
        lst = []
        for token in padded_data[i][0][j][0]:
            lst.append(token_to_index[token])

        # token indices
        review.append(np.array(lst))
        # entitiy
        review.append(token_to_index[data[i][j][1]])
        # attribute
        review.append(token_to_index[data[i][j][2]])
        # polarity
        review.append(data[i][j][3])

        reviews.append(review)

    review_indices.append([reviews,padded_data[i][1]])

In [ ]:
dump_this_dict = {}
for i in range(len(review_indices)):
    dump_this_dict[i] = review_indices[i]

In [21]:
with open('valid_laptops.json', 'wb') as f:
    pickle.dump(dump_this_dict, f)

In [5]:
with open('../processed_data/valid_laptops.json', 'rb') as f:
    data = pickle.load(f)

In [6]:
total = 0
pos = 0
neg = 0
neu = 0

for i in range(len(data)):
    for j in range(data[i][1]):
        if data[i][0][j][3] == 0:
            neg += 1
        elif data[i][0][j][3] == 1:
            neu += 1
        else:
            pos += 1

        total += 1

print(total, pos, neg, neu)

325 149 151 25
